In [1]:
import os
import json
from collections import defaultdict
from statistics import mode
import random

In [2]:
train_metadata_path = "/vision/asomaya1/ego_exo/data/demonstrator_arxiv24_train.json"
with open(train_metadata_path, "r") as f:
    train_data = json.load(f)

#CHANGE val to test if evaluating on test!!
val_metadata_path = "/vision/asomaya1/ego_exo/data/demonstrator_arxiv24_test.json"
with open(val_metadata_path, "r") as f:
    val_data = json.load(f)

is_v2 = True if "24" in val_metadata_path else False
v2_downsampled_rel_path = "downscaled/448"
is_val = True if "val" in val_metadata_path else False

In [3]:
proficiency_mapping = {'Novice': 0, 'Early Expert': 3, 'Intermediate Expert': 2, 'Late Expert': 1}

In [4]:
# Step 1: Collect task names and proficiency scores
task_scores = defaultdict(list)

for entry in train_data:
    task_name = entry['scenario_name']
    proficiency_score = entry['proficiency_score']
    task_scores[task_name].append(proficiency_score)

# Step 2: Process collected data to find most common proficiency score and map to integer
task_mode_score_mapping = {}

for task_name, scores in task_scores.items():
    most_common = mode(scores)
    mode_score_integer = proficiency_mapping[most_common]
    task_mode_score_mapping[task_name] = mode_score_integer

# Step 3: Map video paths to the mode proficiency score for their respective task
video_path_task_mapping = {}

for entry in val_data:
    task_name = entry['scenario_name']
    proficiency_score = entry['proficiency_score']
    video_paths = entry['video_paths']

    for video_path in video_paths.values():
        if is_v2:
            parts = video_path.split("/")
            directory_to_find = "frame_aligned_videos"
            index = parts.index(directory_to_find) + 1
            parts.insert(index, v2_downsampled_rel_path)
            video_path = "/".join(parts)
        if task_name not in task_mode_score_mapping.keys():
            print(task_name)
            video_path_task_mapping[video_path] = random.randint(0,3)
        else:
            video_path_task_mapping[video_path] = task_mode_score_mapping[task_name]

output_filename = 'majority_class_baseline_mapping_full.json'
if is_v2:
    output_filename = 'majority_class_baseline_mapping_full_v2.json'
if is_val:
    output_filename = "val_" + output_filename

with open(output_filename, 'w') as json_file:
    json.dump(video_path_task_mapping, json_file, indent=4)

In [5]:
task_mode_score_mapping

{'Cooking': 0,
 'Basketball': 1,
 'Soccer': 1,
 'Rock Climbing': 0,
 'Music': 1,
 'Dance': 3}